In [6]:
%load_ext  autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import sys
sys.path.append('..')

In [10]:
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from scipy.spatial import distance
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt

seed = 1

In [13]:
# import jtplot module in notebook
from jupyterthemes import jtplot

# choose which theme to inherit plotting style from
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style(theme='oceans16', grid=False, ticks=False, spines=False, context='poster')
# jtplot.style(ticks=True, grid=True, gridlines='--')

# jtplot.reset()


In [14]:
# Load the file
df = pd.read_csv('../data/german_credit_data.csv')
df = df.drop(df.columns[0], axis=1) # remove the index column

# Quantize credit amount, duration and age into 5 bins
amount_series = df.loc[:, 'Credit amount']
df.loc[:, 'Credit amount'] = pd.qcut(amount_series, 5)

duration_series = df.loc[:, 'Duration']
df.loc[:, 'Duration'] = pd.qcut(duration_series, 5)

duration_series = df.loc[:, 'Age']
df.loc[:, 'Age'] = pd.qcut(duration_series, 5)

# Set Job type to object for one-hot encoding
df.loc[:, 'Job'] = df.loc[:, 'Job'].astype(object)

# Perform one-hot encoding
df = pd.get_dummies(df)
# Drop binary features
df = df.drop(columns=['Sex_male', 'Risk_bad'])

# Separate features from targets
df_X = df.iloc[:, :-1]
df_y = df.iloc[:, -1]

print('Examples are represented as {}-dimensional vectors.'.format(df_X.shape[1]))

Examples are represented as 38-dimensional vectors.


In [15]:
# Convert to numpy
X = df_X.values.astype('int8')
y = df_y.values.astype('int8')
print('Shape of X: {}. Shape of y: {}.'.format(X.shape, y.shape))

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=seed)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

Shape of X: (1000, 38). Shape of y: (1000,).


((900, 38), (900,), (100, 38), (100,))

In [16]:
# Fit logistic regression and perform CV
clf = LogisticRegressionCV(
    Cs=21, 
    cv=5, 
    n_jobs=-1, 
    random_state=seed
)
clf.fit(X_train, y_train)

# Get best score and C value
mean_scores = np.mean(clf.scores_[1], axis=0)
best_idx = np.argmax(mean_scores)
best_score = mean_scores[best_idx]
best_C = clf.Cs_[best_idx]

print('Best score is: {:.2f}%. Best C is: {:.4f}.'.format(best_score*100, best_C))
print('Test score is: {:.2f}%.'.format(clf.score(X_test, y_test)*100))

# Best score is: 73.44%. Best C is: 0.1585.
# Test score is: 71.000%.

Best score is: 73.89%. Best C is: 1.0000.
Test score is: 73.00%.


In [23]:
class TransformationWrapper(object):
    amount_start_idx = df_X.columns.get_loc("Credit amount_(249.999, 1262.0]")
    duration_start_idx = df_X.columns.get_loc("Duration_(3.999, 12.0]")
    purpose_start_idx = df_X.columns.get_loc("Purpose_business")
    
    def __init__(self, x, decrease_amount=True, decrease_duration=True):
        self.root = x
        self.decrease_amount = decrease_amount
        self.decrease_duration = decrease_duration
        
        # Slices in the vector for each features
        self.static = self.root[:TransformationWrapper.amount_start_idx]
        self.amount = self.root[TransformationWrapper.amount_start_idx:TransformationWrapper.duration_start_idx]
        self.duration = self.root[TransformationWrapper.duration_start_idx:TransformationWrapper.purpose_start_idx]
        self.purpose = self.root[TransformationWrapper.purpose_start_idx:]
    
    def _get_neighbour(self, x, direction='pos'):
        """Get the neighbouring value in a quantized one-hot feature vector."""
        idx = np.argmax(x)
        if direction == 'pos' and idx != len(x) - 1:
            return np.roll(x, 1).tolist()
        elif direction == 'neg' and idx != 0:
            return np.roll(x, -1).tolist()
        return []

    def _expand_neighbours(self, field, directions=None):
        """Expand neighbouring values of a quantized feature."""
        if directions is None:
            directions = ['pos', 'neg']
            
        child_fields = []
        for direction in directions:
            child_fields.append(self._get_neighbour(field, direction=direction))

        child_fields = [x for x in child_fields if len(x) > 0]
        return np.array(child_fields, dtype='uint8')
    
    def _expand_all(self, field):
        """Expand all values of a categorical feature."""
        child_fields = []
        for i in range(1, len(field)):
            child_fields.append(np.roll(field, i))
        return child_fields

    def expand(self):
        """Generate new transformation."""
        
        children = []
        
        # Expand "credit amount".
        for c in self._expand_neighbours(
                self.amount,
                directions=['pos', 'neg'] if self.decrease_amount else ['pos']):
            child = np.concatenate((self.static, c, self.duration, self.purpose))
            children.append(child)
        
        # Expand "duration".
        for c in self._expand_neighbours(
                self.duration,
                directions=['pos', 'neg'] if self.decrease_duration else ['pos']):
            child = np.concatenate((self.static, self.amount, c, self.purpose))
            children.append(child)
        
        # Expand "purpose".
        for c in self._expand_all(self.purpose):
            child = np.concatenate((self.static, self.amount, self.duration, c))
            children.append(child)
        return children
    
    def __repr__(self):
        return 'TransformationWrapper({})'.format(self.root)

In [24]:
def hash_fn(example):
    return hash(str(example))

def generate_all_transformations(initial_example, transformation_kwargs=None):
    if transformation_kwargs is None:
        transformation_kwargs = {}

    result = []
    closed = set()
    
    queue = [initial_example]
    while queue:
        current_example = queue.pop()            
        current_example_wrapped = TransformationWrapper(
            current_example, **transformation_kwargs)
        for next_example in current_example_wrapped.expand(): 
            h = hash_fn(next_example)
            if h not in closed:
                closed.add(h)
                queue.insert(0, next_example)
                result.append(next_example)
            
    return result

In [40]:
def get_adversarial_transformations(clf, example, target_class=1):
    assert clf.predict([example])[0] != target_class
    transformations = np.array(generate_all_transformations(initial_example))
    classes = clf.predict(transformations)
    adv_transformations = transformations[classes == target_class]
    return adv_transformations



array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)

In [49]:
def cost_fn(initial_example, examples):
    return np.linalg.norm(examples - initial_example, ord=1, axis=1)

example = X[1]
transformations = get_adversarial_transformations(clf, example)
cost_fn(example, transformations)

array([10., 10., 12., 12., 12., 12., 12., 12., 12., 10., 10., 10., 12.,
       12., 12., 12., 12., 12., 12., 10., 12., 12., 12., 12., 12., 12.,
       12., 10., 10., 12., 12., 12., 12., 12., 12., 12., 10., 12., 12.,
       12., 12., 12., 12., 12., 10., 12., 12., 12., 12., 12., 12.,  8.,
       10., 12., 12., 12., 12., 12., 12., 12., 10., 12., 12., 12., 12.,
       12., 12., 12., 10., 12., 12., 12., 12., 12., 12., 12.,  8., 12.,
       12., 12., 12., 12., 12.,  8., 10., 10., 10., 10., 10., 10., 10.,
       10., 12., 12., 12., 12., 12., 12., 12., 10., 12., 12., 12., 12.,
       12., 12., 12.,  8., 12., 12., 12., 12., 12., 12., 12., 10., 10.,
        8., 10., 10., 10., 10., 10., 10., 10., 10., 12., 12., 12., 12.,
       12., 12., 12.,  8., 12., 12., 12., 12., 12., 12., 12., 10., 10.,
       10., 10., 10.,  8., 10., 10., 10., 10., 10., 10., 10.,  8., 12.,
       12., 12., 12., 12., 12., 12., 10., 10., 10., 10., 10., 10.,  6.,
       10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10., 10

In [50]:
def performance_metric_fn(initial_example, examples, target_class=1):
    initial_confidence = clf.predict_proba(initial_example)[0, target_class]
    confidences = clf.predict_proba(examples)[:, target_class]
    return confidences - initial_confidence

In [ ]:
performance_metric_fn(initial_example, )